<a href="https://colab.research.google.com/github/sankhauri/Sales-forecasting-By-Prophet/blob/main/Sales_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sales forecasting

In this project we going to establish a model for forecasting sales of different items from  different stores. Our dataset contains the sales data for 10 stores and 50 items.

# Data set:
Data fields
date - Date of the sale data. There are no holiday effects or store closures.

store - Store ID

item - Item ID

sales - Number of items sold at a particular store on a particular date.

# Data Source
https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data

In [5]:
## getting the dataset

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/train.csv'


Mounted at /content/drive


In [6]:
import pandas as pd
df = pd.read_csv(file_path)
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


In [8]:
## Sorting the dataset according to store number and item number
df_sorted = df.sort_values(by=['store', 'item'], ascending=[True, True])
df_sorted

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [9]:
## Create dataframe for store 1
df_1=df_sorted.iloc[:91300]
df_1

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
896561,2017-12-27,1,50,38
896562,2017-12-28,1,50,52
896563,2017-12-29,1,50,59
896564,2017-12-30,1,50,66


In [10]:
# Pivot without aggregation ( this will put the sales for each items in columns to create a multivariable time series)
pivot_df_1 = df_1.pivot( index='date',columns='item', values='sales')
## Changing column names to item_1,..
columnnames = {}
count = 0
for i in pivot_df_1.columns:

  count += 1

  columnnames[i] = f"store1_item_{count}"

#columnnames
pivot_df_1.rename(columns = columnnames ,inplace = True)
# Rename the dataframe to df_1
df_1=pivot_df_1
df_1.head()  ## date is already in index.

item,store1_item_1,store1_item_2,store1_item_3,store1_item_4,store1_item_5,store1_item_6,store1_item_7,store1_item_8,store1_item_9,store1_item_10,...,store1_item_41,store1_item_42,store1_item_43,store1_item_44,store1_item_45,store1_item_46,store1_item_47,store1_item_48,store1_item_49,store1_item_50
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,13,33,15,10,11,31,25,33,18,37,...,6,21,22,20,37,30,17,21,18,30
2013-01-02,11,43,30,11,6,36,23,37,23,34,...,15,24,27,15,40,30,15,26,10,32
2013-01-03,14,23,14,8,8,18,34,38,25,32,...,5,14,19,11,42,30,5,25,17,25
2013-01-04,13,18,10,19,9,19,36,54,22,45,...,9,22,29,22,49,37,13,26,22,32
2013-01-05,10,34,23,12,8,31,38,51,29,35,...,13,18,34,19,52,28,12,28,15,35
